# Running a CAT based on a synthetic correct/incorrect MIRT model

This example shows how to run a CAT based on a synthetic correct/incorrect
MIRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: DRuleItemCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
dims = 2
using ComputerAdaptiveTesting.DummyData: dummy_full, std_mv_normal, SimpleItemBankSpec, StdModel4PL
using ComputerAdaptiveTesting.MathTraits
using ComputerAdaptiveTesting.Responses: BooleanResponse

TODO: pass in dims

In [3]:
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel4PL(), VectorContinuousDomain(), BooleanResponse()),
    dims;
    num_questions=10,
    num_testees=2
)

(ComputerAdaptiveTesting.ItemBanks.SlipItemBank{ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}}([0.12845117983934407, 0.1658226358061864, 0.2580670754604056, 0.0, 0.03192729066351357, 0.15706015494650374, 0.0884700162713624, 0.0, 0.0, 0.2762506922082394], ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}([0.0, 0.0, 0.0, 0.0, 0.11477265359889677, 0.0, 0.01970663077191055, 0.0, 0.0, 0.0], ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [-0.18880180413801906, 0.4135273676073, 0.6993587150395855, -1.4080460832318566, 1.3303423095655602, -1.3084856072527309, 0.34108006543812125, 0.068586351599637, 0.762364367327994, 1.2038656354134594], [1.102675886190874 1.1884495249189135 … 1.2730190941953097 0.8842758927319168; 1.064498628120071

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [4]:
max_questions = 9
integrator = CubaIntegrator([-6.0, -6.0], [6.0, 6.0], CubaVegas(); rtol=1e-2)
ability_estimator = MeanAbilityEstimator(PriorAbilityEstimator(std_mv_normal(dims)), integrator)
rules = CatRules(
    ability_estimator,
    DRuleItemCriterion(ability_estimator),
    FixedItemsTerminationCondition(max_questions)
)

CatRules
  next_item: ComputerAdaptiveTesting.NextItemRules.ItemStrategyNextItemRule{ComputerAdaptiveTesting.NextItemRules.ExhaustiveSearch1Ply, ComputerAdaptiveTesting.NextItemRules.DRuleItemCriterion{ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integrators.CubaIntegrator{ComputerAdaptiveTesting.Integrators.CubaVegas, Base.Pairs{Symbol, Float64, Tuple{Symbol}, NamedTuple{(:rtol,), Tuple{Float64}}}}}}}}
  termination_condition: ComputerAdaptiveTesting.TerminationConditions.FixedItemsTerminationCondition
  ability_estimator: ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integra

XXX: We shouldn't need to specify xs here since the distributions are not used -- rework

In [5]:
points = 3
xs = repeat(range(-2.5, 2.5, length=points)', dims, 1)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator, abilities)
for testee_idx in axes(responses, 2)
    @debug "Running for testee" testee_idx
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[:, testee_idx]
    abs_err = sum(abs.(θ .- true_θ))
    @info "convergence" true_θ θ abs_err
end

┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     -1.8847634952213979
│     -1.8408457628127677
│   θ =
│    2-element Vector{Float64}:
│     -1.3621063181522144
│     -1.4882712136820582
└   abs_err = 0.875231726199893
┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     0.7392403346486828
│     0.4904664840993218
│   θ =
│    2-element Vector{Float64}:
│     0.7986262947856597
│     0.6233620279091769
└   abs_err = 0.19228150394683208


Make a plot showing how the estimated value converges during the CAT.

In [6]:
conv_lines_fig = ability_convergence_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*